In [1]:
from omegaconf import OmegaConf

config = OmegaConf.load("config.yaml")
config

{'backbone_path': 'siglip_checkpoints', 'training': True, 'val_dataset': False, 'temperature': 0.02, 'img_aug': True, 'cmt_depth': 4, 'lr_factor': 5.0, 'MLM': True, 'loss_names': ['sdm', 'id', 'mlm'], 'mlm_loss_weight': 1.0, 'id_loss_weight': 1.0, 'img_size': [384, 128], 'interpolate_pos_encoding': True, 'stride_size': 16, 'text_length': 64, 'vocab_size': 250001, 'optimizer': 'PagedAdamW8bit', 'lr': 1e-05, 'bias_lr_factor': 2.0, 'momentum': 0.9, 'weight_decay': 4e-05, 'weight_decay_bias': 0.0, 'alpha': 0.9, 'beta': 0.999, 'num_epochs': 120, 'milestones': [20, 50], 'gamma': 0.1, 'warmup_factor': 0.1, 'warmup_epochs': 5, 'warmup_method': 'linear', 'lrscheduler': 'cosine', 'target_lr': 0, 'power': 0.9, 'dataset_name': 'VN3K', 'sampler': 'random', 'num_intance': 4, 'root_dir': '.', 'batch_size': 8, 'test_batch_size': 64, 'num_workers': 4, 'accumulate_grad_batches': 2}

In [2]:
from lightning_data import IRRADataModule

dm = IRRADataModule(config)

/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VN3K


In [3]:
dm.setup()

In [4]:
train = dm.train_set

In [5]:
print("Image shape:", train[0]['images'].shape)
print("Caption shape:", train[0]['caption_input']['input_ids'].shape)

Image shape: torch.Size([3, 384, 128])
Caption shape: torch.Size([1, 64])


In [6]:
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

In [7]:
train_loader.batch_size

8

In [8]:
config.num_epochs

120

In [9]:
import lightning as L
from lighning_models import LitIRRA

model = LitIRRA(config, img_loader=test_loader[0], text_loader=test_loader[1], num_classes=dm.num_classes)

In [10]:
model.model

IRRA(
  (text_model): SiglipTextModel(
    (text_model): SiglipTextTransformer(
      (embeddings): SiglipTextEmbeddings(
        (token_embedding): Embedding(250001, 768)
        (position_embedding): Embedding(64, 768)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-11): 12 x SiglipEncoderLayer(
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_features=768, b

In [11]:
model.model(next(iter(train_loader)))

{'temperature': tensor(0.0200),
 'sdm_loss': tensor(24.4294, grad_fn=<AddBackward0>),
 'id_loss': tensor(7.6044, grad_fn=<MulBackward0>),
 'img_acc': tensor(0.),
 'txt_acc': tensor(0.),
 'mlm_loss': tensor(12.4970, grad_fn=<MulBackward0>),
 'mlm_acc': tensor(0.)}

In [13]:
test_img = next(iter(train_loader))['images']
test_caption = next(iter(train_loader))['caption_input']

In [18]:
model.model.encode_image(test_img).shape

torch.Size([8, 768])

In [17]:
model.model.encode_text(test_caption).shape

torch.Size([8, 768])

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(monitor='eval_score')

In [21]:
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = L.Trainer(precision="bf16-mixed", logger=logger, max_epochs=config.num_epochs, accumulate_grad_batches=16)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [22]:
trainer.predict(model, test_loader)

MisconfigurationException: `Trainer.predict` requires `forward` method to run.

In [13]:
trainer.fit(model, train_dataloaders=train_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | IRRA | 595 M 
-------------------------------
595 M     Trainable params
0         Non-trainable params
595 M     Total params
2,382.905 Total estimated model params size (MB)


Using 5.0 times learning rate for random init module 
Epoch 0:   0%|          | 2/8604 [00:02<2:49:18,  0.85it/s, v_num=1, temperature_step=0.020, sdm_loss_step=10.70, id_loss_step=7.600, img_acc_step=0.000, txt_acc_step=0.000, mlm_loss_step=12.40, mlm_acc_step=0.000, total_loss_step=30.70]

OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 11.55 GiB total capacity; 9.62 GiB already allocated; 728.62 MiB free; 10.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def collate(batch):
    # Initialize dictionary to hold batched data
    batched_data = {}

    # Get all keys from the first item in the batch to identify field names
    keys = batch[0].keys()

    for key in keys:
        if isinstance(batch[0][key], dict):
            # If the field is a nested dictionary, process each sub-field
            sub_keys = batch[0][key].keys()
            batched_data[key] = {}
            for sub_key in sub_keys:
                sub_values = [item[key][sub_key] for item in batch]
                if isinstance(sub_values[0], torch.Tensor):
                    # Concatenate tensors
                    batched_data[key][sub_key] = torch.cat(sub_values, dim=0)
                else:
                    # Assume list of primitive types and convert to tensor
                    batched_data[key][sub_key] = torch.tensor(sub_values, dtype=torch.int64)
        else:
            # If the field is not a nested dictionary, process directly
            values = [item[key] for item in batch]
            if isinstance(values[0], torch.Tensor):
                # Stack tensors if they are of the same size
                batched_data[key] = torch.stack(values)
            else:
                # Assume list of primitive types and convert to tensor
                batched_data[key] = torch.tensor(values, dtype=torch.int64)

    return batched_data

train = dm.train_set

In [ ]:
train[2]

{'pids': 0,
 'image_ids': 0,
 'images': tensor([[[ 0.0617,  0.0763,  0.0471,  ..., -1.7923, -1.7923, -1.7923],
          [ 0.1055,  0.1055,  0.0617,  ..., -1.7923, -1.7923, -1.7923],
          [ 0.0325,  0.0471,  0.0325,  ..., -1.7923, -1.7923, -1.7923],
          ...,
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923]],
 
         [[-0.6415, -0.6565, -0.6865,  ..., -1.7521, -1.7521, -1.7521],
          [-0.5965, -0.6265, -0.6715,  ..., -1.7521, -1.7521, -1.7521],
          [-0.6565, -0.6715, -0.7016,  ..., -1.7521, -1.7521, -1.7521],
          ...,
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521]],
 
         [[-0.3853, -0.3711, -0.3995,  ..., -1.4802, -1.4802,

In [1]:
from model.siglip.tokenization_siglip import SiglipTokenizer

tokenizer = SiglipTokenizer.from_pretrained("/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints")

/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(tokenizer)

250100

In [3]:
mask_token = tokenizer.convert_ids_to_tokens(250000)
tokenizer.add_special_tokens({"mask_token": mask_token})

0

'<mask>'